In [78]:
import requests
from duckduckgo_search import DDGS
from time import sleep
import pandas as pd
from pydantic import BaseModel, ValidationError
from time import time

In [46]:
def search_news_by_keywords(keyword, persona_name, max_results=10000):
    search_term = keyword 
    with DDGS() as ddgs:
        results = ddgs.news(
            keywords=search_term,
            region="es-es",
            safesearch="Moderate",
            timelimit="w",
            max_results=max_results
        )
        return [{"persona": persona_name, "query": search_term, **r} for r in results]


In [47]:
persona_keywords = {
    "Laura Gil": [
        "inteligencia artificial", "datos", "transformación digital",
        "industria 4.0", "automatización", "cultura de innovación"
    ],
    "Fede Segarra": [
        "comunicación corporativa", "reputación de marca", "PR", 
        "estrategias de comunicación", "imagen pública"
    ],
    "Elísabeth Hernández": [
        "recursos humanos", "RSC", "desarrollo del talento", 
        "formación corporativa", "comunicación interna", "voluntariado"
    ],
    "Jaume Alemany": [
        "marketing", "branding", "consumo", "bebidas", 
        "innovación de producto", "campañas publicitarias"
    ],
    "Ricardo Lechuga": [
        "transformación cultural", "talento interno", "digitalización RRHH", 
        "engagement de empleados", "liderazgo cercano"
    ],
    "Jorge Villavecchia": [
        "liderazgo empresarial", "estrategia corporativa", "sostenibilidad", 
        "transformación de negocios", "expansión internacional"
    ],
    "Salvador Martínez": [
        "finanzas sostenibles", "crecimiento responsable", 
        "decisiones basadas en datos", "ética empresarial"
    ],
    "Jofre Riera": [
        "patrocinios deportivos", "marketing cultural", 
        "experiencias de marca", "impacto social"
    ]
}


In [51]:
all_results = []
for persona, keywords in persona_keywords.items():
    keyword_results = []
    for keyword in keywords:
        results = search_news_by_keywords(keyword, persona)
        keyword_results.extend(results)
        print(f"Results for {persona} on '{keyword}': {len(results)} found.")
        print(f"Sleeping for 20 seconds to avoid rate limiting...")
        sleep(20)
        break 
        
    all_results.extend(keyword_results)

Results for Laura Gil on 'inteligencia artificial': 52 found.
Sleeping for 20 seconds to avoid rate limiting...
Results for Fede Segarra on 'comunicación corporativa': 26 found.
Sleeping for 20 seconds to avoid rate limiting...
Results for Elísabeth Hernández on 'recursos humanos': 57 found.
Sleeping for 20 seconds to avoid rate limiting...
Results for Jaume Alemany on 'marketing': 83 found.
Sleeping for 20 seconds to avoid rate limiting...
Results for Ricardo Lechuga on 'transformación cultural': 35 found.
Sleeping for 20 seconds to avoid rate limiting...
Results for Jorge Villavecchia on 'liderazgo empresarial': 47 found.
Sleeping for 20 seconds to avoid rate limiting...
Results for Salvador Martínez on 'finanzas sostenibles': 32 found.
Sleeping for 20 seconds to avoid rate limiting...
Results for Jofre Riera on 'patrocinios deportivos': 30 found.
Sleeping for 20 seconds to avoid rate limiting...


In [70]:
df_results = pd.DataFrame(all_results)
df_results.head()

,persona,query,date,title,body,url,image,source
0,Laura Gil,inteligencia artificial,2025-06-20T11:39:00+00:00,León XIV quiere una inteligencia artificial qu...,"Desde ayer hasta hoy, 20 de junio, Roma está a...",https://www.vidanuevadigital.com/2025/06/20/le...,https://www.vidanuevadigital.com/wp-content/up...,Vida Nueva
1,Laura Gil,inteligencia artificial,2025-06-20T11:42:00+00:00,Esta es la ciudad de España donde viven las pe...,Esta tecnología indica que no existe en España...,https://www.abc.es/recreo/ciudad-espana-viven-...,https://s1.abcstatics.com/abc/www/multimedia/r...,ABC
2,Laura Gil,inteligencia artificial,2025-06-20T08:13:00+00:00,El español impulsa la inteligencia artificial ...,"Proyectos innovadores, startups y colaboración...",https://www.infobae.com/tecno/2025/06/20/el-es...,https://www.infobae.com/resizer/v2/OPPFAUUDPNF...,Infobae
3,Laura Gil,inteligencia artificial,2025-06-20T12:55:13+00:00,El Teatro Mariinski estrena la primera ópera c...,El estreno de la obra ha sido el suceso cultur...,https://www.msn.com/es-es/entretenimiento/musi...,https://fotografias.larazon.es/clipping/cmsima...,La Razón
4,Laura Gil,inteligencia artificial,2025-06-20T13:16:00+00:00,Inteligencia Artificial En Espanol,El sitio de noticias en español más leído en e...,https://www.infobae.com/tag/inteligencia-artif...,https://www.infobae.com/pf/resources/images/fa...,Infobae


In [71]:
len(df_results)

362

In [87]:
class RawArticle(BaseModel):
    run_id: str
    source_name: str
    source_url: str
    article_id: str
    article_date: str
    article_title: str
    article_url: str
    article_body: str
    article_image_url: str
    article_language: str
    crawled_at: str
    ddgs_search_query: str
    query_original_personas: list[str]

In [88]:
runid = "DEVRUN"

In [103]:
df_results = pd.DataFrame(all_results)
df_results.head()

agg_clause = {col : 'first' for col in df_results.columns}
agg_clause.pop('url')
agg_clause['persona'] = lambda x: list(x)
agg_clause['query'] = lambda x: list(x)
df_results = df_results.groupby('url').agg(agg_clause).reset_index()

df_results["source_url"] = df_results["url"].apply(lambda x: "/".join(x.split("/")[:3]) if pd.notna(x) else None)
df_results["source"] = df_results["source"].apply(lambda x: x.replace(" ", "_").lower() if pd.notna(x) else None)
df_results["article_id"] = df_results["source"].apply(lambda x: x + "_" + str(time()).replace(".", "") if pd.notna(x) else None)
df_results.dropna(subset=["source_url", "source", "article_id"], inplace=True)
df_results["crawled_at"] = pd.to_datetime("now").isoformat()
df_results["article_language"] = "es"  
df_results["run_id"] = runid

df_results.rename(columns={
    "title": "article_title",
    "url": "article_url",
    "body": "article_body",
    "image": "article_image_url",
    "source": "source_name",
    "query": "ddgs_search_query",
    "persona": "query_original_personas",
    "date" : "article_date"
}, inplace=True)

df_results.head()

,article_url,query_original_personas,ddgs_search_query,article_date,article_title,article_body,article_image_url,source_name,source_url,article_id,crawled_at,article_language,run_id
0,https://abcnoticias.mx/local/2025/6/19/asegura...,[Jorge Villavecchia],[liderazgo empresarial],2025-06-20T05:24:00+00:00,Asegura Estado que en NL hay 3 millones de tra...,El estado cuenta con una Población Económicame...,https://abcnoticias.mx/u/fotografias/m/2025/6/...,abc_noticias,https://abcnoticias.mx,abc_noticias_17504333710091262,2025-06-20T17:29:31.010764,es,DEVRUN
1,https://acento.com.do/opinion/americo-lugo-con...,[Ricardo Lechuga],[transformación cultural],2025-06-19T04:03:00+00:00,"Américo Lugo, conciencia de nación",La democracia dominicana no alcanza su mayor e...,https://media.acento.com.do/media/storage02/up...,acento,https://acento.com.do,acento_17504333710091338,2025-06-20T17:29:31.010764,es,DEVRUN
2,https://amprensa.com/2025/06/una-cerveza-en-la...,[Jofre Riera],[patrocinios deportivos],2025-06-14T13:19:00+00:00,Una cerveza en la cancha: Michelob se suma al ...,-La marca fortalece su vínculo con el deporte ...,https://cdn.amprensa.com/785ac538-0f24-40f0-8e...,amprensa.com,https://amprensa.com,amprensa.com_17504333710091372,2025-06-20T17:29:31.010764,es,DEVRUN
3,https://blogs.elconfidencial.com/economia/trib...,"[Fede Segarra, Jorge Villavecchia]","[comunicación corporativa, liderazgo empresarial]",2025-06-15T03:00:00+00:00,El nuevo liderazgo empresarial se mide en inta...,Vivimos en un contexto marcado por la volatili...,https://images.ecestaticos.com/9RnhwFbvwGgFK-w...,blogs_en_el_confidencial,https://blogs.elconfidencial.com,blogs_en_el_confidencial_17504333710091393,2025-06-20T17:29:31.010764,es,DEVRUN
4,https://blogs.elconfidencial.com/mercados/el-a...,[Jorge Villavecchia],[liderazgo empresarial],2025-06-20T03:00:00+00:00,Tu decisión más importante: quién construye a ...,El último informe del World Economic Forum rev...,https://images.ecestaticos.com/lI9rSLCtTt6IrCq...,blogs_en_el_confidencial,https://blogs.elconfidencial.com,blogs_en_el_confidencial_1750433371009141,2025-06-20T17:29:31.010764,es,DEVRUN


In [101]:
# convert to RawArticle model
raw_articles = []
for _, row in df_results.iterrows():
    try:
        article = RawArticle(
            run_id=row["run_id"],
            source_name=row["source_name"],
            source_url=row["source_url"],
            article_id=row["article_id"],
            article_date=row["article_date"],
            article_title=row["article_title"],
            article_url=row["article_url"],
            article_body=row["article_body"],
            article_image_url=row["article_image_url"],
            article_language=row["article_language"],
            crawled_at=row["crawled_at"],
            ddgs_search_query=row["ddgs_search_query"],
            query_original_personas=row["query_original_personas"]
        )
        raw_articles.append(article)
    except ValidationError as e:
        print(f"Validation error for row {row['article_id']}: {e}")

In [102]:
raw_articles

[RawArticle(run_id='DEVRUN', source_name='abc_noticias', source_url='https://abcnoticias.mx', article_id='abc_noticias_17504326268107831', article_date='2025-06-20T05:24:00+00:00', article_title='Asegura Estado que en NL hay 3 millones de trabajadores formales', article_url='https://abcnoticias.mx/local/2025/6/19/asegura-estado-que-en-nl-hay-millones-de-trabajadores-formales-252475.html', article_body='El estado cuenta con una Población Económicamente Activa cercana a los 3 millones de personas, con una tasa de desempleo de apenas 2.8% y una informalidad laboral del 32.5%, la más baja del país, lo que se traduce en mayor acceso a seguridad social, prestaciones laborales y estabilidad económica para los trabajadores.', article_image_url='https://abcnoticias.mx/u/fotografias/m/2025/6/19/f1280x720-302753_434428_5050.png', article_language='es', crawled_at='2025-06-20T17:17:06.811882', ddgs_search_query='liderazgo empresarial', query_original_personas=['Jorge Villavecchia']),
 RawArticle(r